## Importing Libraries:

In [1]:
# Importing Libraries: 
import pandas as pd 
import sqlite3 

## Connecting to the DBs:

In [2]:
conn = sqlite3.connect("Data Sources/Price_Data.sqlite3")
c = conn.cursor()

###
c.execute("""SELECT name FROM sqlite_master 
             WHERE 
             type ='table' AND 
             name NOT LIKE 'sqlite_%';""")
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,name
0,bevprice
1,foodprice
2,zonenumbers
3,itemnumbers


In [3]:
# Connecting to the second DB:

conn2 = sqlite3.connect("Data Sources/Register_Data.sqlite3")
c2 = conn2.cursor()

###
c2.execute("""SELECT name FROM sqlite_master 
             WHERE 
             type ='table' AND 
             name NOT LIKE 'sqlite_%';""")
df = pd.DataFrame(c2.fetchall())
df.columns = [i[0] for i in c2.description]
df

,name
0,currentprices


In [4]:
# Some adjustments to the currentprices sheet to make it easier to use:

c2.execute("""SELECT ItemNumber || "-" || PriceZoneNumber AS ItemZone, *
              FROM currentprices
              WHERE PriceZoneType = 'Bev'
              OR PriceZoneType = 'Food';""")
df_reg = pd.DataFrame(c2.fetchall())
df_reg.columns = [i[0] for i in c2.description]
# I excluded Merch and Drink as the instructions only call for comparisons between 
# I added 'Item Zone as a better way to do inner joins on Item Number and Zone Number' when it comes to comparing this table to the Price Database

###
display(df_reg.head())
df_reg.shape
df_reg.PriceZoneType.value_counts()
# I excluded Merch and Drink as the instructions only call for comparisons between 
# I added 'Item Zone as a better way to do inner joins on Item Number and Zone Number' when it comes to comparing this table to the Price Database

,ItemZone,ItemNumber,PriceZoneNumber,PriceZoneType,Price
0,123457-2,123457,2,Bev,1.55
1,123457-4,123457,4,Bev,2.15
2,123457-14,123457,14,Bev,2.25
3,123457-16,123457,16,Bev,2.35
4,123457-17,123457,17,Bev,2.15


Bev     386205
Food    135771
Name: PriceZoneType, dtype: int64

In [5]:
# Adding currentprices to the main db
df_reg.to_sql("currentprices", conn, index = False)

In [6]:
# Simplifying the Price DataBase into one 'source of truth' table:
# First I'm combining the foodprice and bevprice tables (each joined with zonenumbers for ZoneNumber) so I can do an inner join. . . 
# . . . with itemnumbers so I can create the same field, ItemZone, as I created in the currentprices table for the joins. . . 
#. . . necessary for the main tables I'll be creating. It'll act as a key.
# The following table has everything needed from the Price DB side of the equation. All that's left is joins with the Register DB.

c.execute("""WITH foodbev AS (
             SELECT f.TierSize, f.FoodZoneName AS ZoneName, 
             z.ZoneNumber, f.Price
             FROM foodprice f
             INNER JOIN zonenumbers z
             ON f.FoodZoneName = z.ZoneName
             
             UNION ALL
             
             SELECT b.TierSize, b.BevZoneName AS ZoneName, z.ZoneNumber, b.Price
             FROM bevprice b
             INNER JOIN zonenumbers z
             ON b.BevZoneName = z.ZoneName)
             
                 SELECT CAST(i.ItemNumber AS INT) || "-" || fb.ZoneNumber AS ItemZone,
                 i.ItemNumber, fb.ZoneName, fb.ZoneNumber, fb.Price as PriceData, i.FoodOrBev
                 FROM itemnumbers i
                 INNER JOIN foodbev fb
                 ON i.TierSize = fb.TierSize;""")
df_price_data = pd.DataFrame(c.fetchall())
df_price_data.columns = [i[0] for i in c.description]

###
display(df_price_data.head())
df_price_data.shape

# I added 'Item Zone here as well for the reasons noted above

,ItemZone,ItemNumber,ZoneName,ZoneNumber,PriceData,FoodOrBev
0,123457-4,000000123457,Store Group 0,4,2.15,Bev
1,123457-16,000000123457,Store Group 1,16,2.35,Bev
2,123457-145,000000123457,Store Group 10,145,2.05,Bev
3,123457-151,000000123457,Store Group 11,151,2.15,Bev
4,123457-152,000000123457,Store Group 12,152,2.55,Bev


(189986, 6)

In [7]:
# Since this is the table I'll be referencing the most, I'll add it to the database
# I'm naming it 'pricedata' to stick to naming conventions and indicate that this is the source of truth for the price db
df_price_data.to_sql("pricedata", conn, index = False)
# Now I have all the necessary tables ready so I can move on to the questions

# Q1:
“Price_Deltas.csv”: a csv that contains any beverage or food products that have a price mismatch between Price_Data and Register_Data

In [9]:
# Table 1:
c.execute("""SELECT p.ItemNumber, r.PriceZoneNumber, r.PriceZoneType, r.Price AS PriceRegister,
             p.PriceData, 
             ABS(p.PriceData - r.Price) AS PriceDelta
             FROM pricedata p
             INNER JOIN currentprices r
             ON 
             p.ItemZone = r.ItemZone
             WHERE PriceDelta > 0;""")
df1 = pd.DataFrame(c.fetchall())
df1.columns = [i[0] for i in c.description]
display(df1.head())
df1.shape
# For this table, I used an inner join so that I'm only looking at rows that contain values in both tables so I can be sure there's a mismatch.

,ItemNumber,PriceZoneNumber,PriceZoneType,PriceRegister,PriceData,PriceDelta
0,000011132803,4,Bev,2.75,2.95,0.2
1,000011132803,16,Bev,3.15,3.45,0.3
2,000011132803,145,Bev,2.75,2.95,0.2
3,000011132803,151,Bev,2.75,2.95,0.2
4,000011132803,152,Bev,2.75,3.45,0.7


(2399, 6)

In [11]:
# CSV:
df1.to_csv(r'/Users/Thomas/Desktop/Outputs/CSVs/Price_Deltas.csv', index = False)

# Q2:
“Missing_From_Price_Data.csv”: a csv that contains any beverage or food products that have a price listed in the Register_Data, but not present in the Price_Data

In [12]:
# Table 2:

c.execute("""SELECT r.ItemNumber, r.PriceZoneNumber, r.PriceZoneType, r.Price AS PriceRegister,
             p.PriceData, 
             ABS(p.PriceData - r.Price) AS PriceDelta
             FROM currentprices r
             LEFT JOIN pricedata p
             ON 
             p.ItemZone = r.ItemZone
             WHERE p.PriceData IS NULL;""")
df2 = pd.DataFrame(c.fetchall())
df2.columns = [i[0] for i in c.description]
display(df2.head())
df2.shape
# For this second table, I used a left join starting with Register data but specified that I only want rows that have a price missing from the Price DB

,ItemNumber,PriceZoneNumber,PriceZoneType,PriceRegister,PriceData,PriceDelta
0,123457,2,Bev,1.55,None,None
1,123457,14,Bev,2.25,None,None
2,123457,56,Bev,1.40,None,None
3,123457,72,Bev,2.05,None,None
4,123457,77,Bev,1.55,None,None


(345962, 6)

In [13]:
# CSV:
df2.to_csv(r'/Users/Thomas/Desktop/Outputs/CSVs/Missing_From_Price_Data.csv', index = False)

# Q3:
“Missing_From_Register_Data.csv”: a csv that contains any beverage or food products that have a price listed in the Price_Data, but not present in the Register_Data

In [14]:
# Table 3:
c.execute("""SELECT p.ItemNumber, p.ZoneNumber, p.FoodOrBev AS PriceZoneType, 
             r.Price AS PriceRegister,
             p.PriceData, 
             ABS(p.PriceData - r.Price) AS PriceDelta
             FROM pricedata p
             LEFT JOIN currentprices r
             ON 
             p.ItemZone = r.ItemZone
             WHERE r.Price IS NULL;""")
df3 = pd.DataFrame(c.fetchall())
df3.columns = [i[0] for i in c.description]
display(df3.head())
df3.shape
# For the third table, I used a left join starting with Price data but specified that I only want rows that have a price missing from the Register DB

,ItemNumber,ZoneNumber,PriceZoneType,PriceRegister,PriceData,PriceDelta
0,000011132803,40,Bev,None,3.45,None
1,000011243447,44,Food,None,0.75,None
2,000011243447,9,Food,None,0.75,None
3,000011243447,11,Food,None,0.75,None
4,000011243447,12,Food,None,0.75,None


(13970, 6)

In [15]:
# CSV:
df3.to_csv(r'/Users/Thomas/Desktop/Outputs/CSVs/Missing_From_Register_Data.csv', index = False)

# Audit DB:

In [16]:
conn3 = sqlite3.connect("Audits.sqlite3")
c3 = conn3.cursor()

In [17]:
df1.to_sql("Price_Deltas", conn3, index = False)
df2.to_sql("Missing_From_Price_Data", conn3, index = False)
df3.to_sql("Missing_From_Register_Data", conn3, index = False)

In [18]:
c3.execute("""SELECT name FROM sqlite_master 
             WHERE 
             type ='table' AND 
             name NOT LIKE 'sqlite_%';""")
df = pd.DataFrame(c3.fetchall())
df.columns = [i[0] for i in c3.description]
df
# Looks like it's all there!

,name
0,Price_Deltas
1,Missing_From_Price_Data
2,Missing_From_Register_Data


Thanks for reading!

-TB